In [4]:
# @title 1. Setup Environment and Mount Google Drive
# This cell sets up necessary libraries and connects to your Google Drive.

import os
import shutil
import random
from tqdm import tqdm # For progress bars
import zipfile
import requests # For Pythonic download alternative (not used directly here, but good to have)

# Install split-folders library - essential for stratified splitting
!pip install split-folders tqdm

from splitfolders import ratio

# Mount Google Drive (REQUIRED for persistent storage of large files)
from google.colab import drive
drive.mount('/content/drive')

print("Environment setup complete and Google Drive mounted.")

# --- DEFINE YOUR GOOGLE DRIVE PROJECT DATA ROOT ---
# This MUST match the path you set in 'drive_setup_project_data_dirs.ipynb'
GOOGLE_DRIVE_PROJECT_ROOT = '/content/drive/MyDrive/AgroAI_Project_Data' # <--- ENSURE THIS MATCHES YOUR SETUP!
if not os.path.exists(GOOGLE_DRIVE_PROJECT_ROOT):
    raise FileNotFoundError(f"Google Drive project root not found: {GOOGLE_DRIVE_PROJECT_ROOT}. Please run 'drive_setup_project_data_dirs.ipynb' first.")
print(f"Google Drive project root (for data storage): {GOOGLE_DRIVE_PROJECT_ROOT}")

# --- DEFINE PROJECT-SPECIFIC DATA PATHS WITHIN GOOGLE DRIVE ---
# These paths are where Module 1's large data and models will be stored persistently.
MODULE1_DRIVE_DATA_DIR = os.path.join(GOOGLE_DRIVE_PROJECT_ROOT, 'module1_edge_ai', 'data')
MODULE1_DRIVE_MODELS_DIR = os.path.join(GOOGLE_DRIVE_PROJECT_ROOT, 'module1_edge_ai', 'trained_models')

# Ensure these directories exist (they should, if drive_setup_project_data_dirs.ipynb was run)
os.makedirs(MODULE1_DRIVE_DATA_DIR, exist_ok=True)
os.makedirs(MODULE1_DRIVE_MODELS_DIR, exist_ok=True) # Ensure models dir is also there
print(f"Module 1 data will be stored persistently in Drive at: {MODULE1_DRIVE_DATA_DIR}")
print(f"Module 1 models will be stored persistently in Drive at: {MODULE1_DRIVE_MODELS_DIR}")

# Define where your raw PlantVillage ZIP will be downloaded *to* in Google Drive
PLANTVILLAGE_ZIP_PATH_DRIVE = os.path.join(MODULE1_DRIVE_DATA_DIR, 'plantvillage_dataset.zip')

# Define the source URL for the PlantVillage dataset
PLANTVILLAGE_DOWNLOAD_URL = "https://storage.googleapis.com/plantdata/PlantVillage.zip"

# Define the directory names for extracted and subset data within your Drive module data folder
EXTRACTED_DATA_DIR = os.path.join(MODULE1_DRIVE_DATA_DIR, 'PlantVillage_Raw')
SUBSET_OUTPUT_DIR = os.path.join(MODULE1_DRIVE_DATA_DIR, 'PlantVillage_Subset')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Environment setup complete and Google Drive mounted.
Google Drive project root (for data storage): /content/drive/MyDrive/AgroAI_Project_Data
Module 1 data will be stored persistently in Drive at: /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data
Module 1 models will be stored persistently in Drive at: /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/trained_models


In [9]:
# @title 2.1. Clone PlantVillage Dataset from Git Repository
# This cell clones the dataset from the specified Git repository instead of downloading a zip.

GIT_REPO_URL = "https://github.com/gabrieldgf4/PlantVillage-Dataset.git"
REPO_CLONE_PATH = os.path.join(MODULE1_DRIVE_DATA_DIR, 'PlantVillage_GitClone')

print(f"\n--- Attempting to clone PlantVillage dataset from Git: {GIT_REPO_URL} ---")

if os.path.exists(REPO_CLONE_PATH):
    print(f"Existing clone found at {REPO_CLONE_PATH}. Removing for fresh clone...")
    shutil.rmtree(REPO_CLONE_PATH) # Clear previous clone
os.makedirs(REPO_CLONE_PATH, exist_ok=True) # Ensure parent directory exists

# Clone the repository
!git clone {GIT_REPO_URL} "{REPO_CLONE_PATH}"

print("Git cloning complete.")

# Set the ORIGINAL_DATA_ROOT to the location of the cloned data
# Assuming the images are directly within a subdirectory of the cloned repo, e.g., 'PlantVillage-Dataset/dataset'
# You might need to inspect the cloned directory structure and adjust this path
# For now, let's assume the class folders are directly within the cloned repo root or a subdirectory
# We'll need to verify this after execution.
# Let's make a reasonable guess based on typical repository structure.
# If the data is in a folder named 'dataset' within the cloned repo:
ASSUMED_DATA_SUBDIR = 'dataset'
ORIGINAL_DATA_ROOT = os.path.join(REPO_CLONE_PATH, ASSUMED_DATA_SUBDIR)

# If the data is directly in the cloned repo root, use this instead:
# ORIGINAL_DATA_ROOT = REPO_CLONE_PATH


print(f"Assumed original dataset root for splitting (containing class folders): {ORIGINAL_DATA_ROOT}")

# Basic check to see if the assumed path exists and has content
if not os.path.exists(ORIGINAL_DATA_ROOT) or not os.listdir(ORIGINAL_DATA_ROOT):
     print(f"WARNING: Assumed data root '{ORIGINAL_DATA_ROOT}' not found or is empty.")
     print("Please inspect the contents of the cloned repository at:")
     print(REPO_CLONE_PATH)
     print("And manually set the correct ORIGINAL_DATA_ROOT path based on the actual directory structure.")
else:
    print("Assumed data root seems valid. Proceeding with splitting (Cell 3).")

# Note: The splitting cell (Cell 3) still expects ORIGINAL_DATA_ROOT to point to the directory
# containing the class subfolders. You might need to manually update the path in Cell 3
# if the structure of the cloned repository is different than expected.


--- Attempting to clone PlantVillage dataset from Git: https://github.com/gabrieldgf4/PlantVillage-Dataset.git ---
Existing clone found at /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_GitClone. Removing for fresh clone...
Cloning into '/content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_GitClone'...
remote: Enumerating objects: 54511, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 54511 (delta 36), reused 0 (delta 0), pack-reused 54403 (from 1)
Receiving objects: 100% (54511/54511), 806.27 MiB | 12.75 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Updating files: 100% (54306/54306), done.
Git cloning complete.
Assumed original dataset root for splitting (containing class folders): /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_GitClone/dataset
Please inspect the contents of the cloned repository at:
/content/drive/MyDrive/Ag

In [10]:
# @title 2. Download and Extract PlantVillage Dataset Directly to Google Drive
# This cell handles getting the raw PlantVillage dataset and saving it persistently.

# print(f"\n--- Attempting to download PlantVillage dataset to Google Drive: {PLANTVILLAGE_ZIP_PATH_DRIVE} ---")

# if not os.path.exists(PLANTVILLAGE_ZIP_PATH_DRIVE):
#     print(f"Downloading from: {PLANTVILLAGE_DOWNLOAD_URL}")
#     # Using !wget for direct download to Google Drive path
#     !wget -q {PLANTVILLAGE_DOWNLOAD_URL} -O "{PLANTVILLAGE_ZIP_PATH_DRIVE}"
#     print("Download complete.")
# else:
#     print("PlantVillage zip already exists in Google Drive. Skipping download.")

# # Verify that the ZIP file exists before proceeding
# if not os.path.exists(PLANTVILLAGE_ZIP_PATH_DRIVE):
#     raise FileNotFoundError(f"ERROR: PlantVillage dataset ZIP not found at {PLANTVILLAGE_ZIP_PATH_DRIVE} after download attempt.")

# # Extract the dataset
# print(f"\n--- Extracting dataset from {PLANTVILLAGE_ZIP_PATH_DRIVE} to {EXTRACTED_DATA_DIR} ---")
# if os.path.exists(EXTRACTED_DATA_DIR):
#     print(f"Existing extraction found at {EXTRACTED_DATA_DIR}. Deleting for fresh extraction...")
#     shutil.rmtree(EXTRACTED_DATA_DIR) # Clear previous extraction for idempotence
# os.makedirs(EXTRACTED_DATA_DIR, exist_ok=True) # Recreate the directory

# with zipfile.ZipFile(PLANTVILLAGE_ZIP_PATH_DRIVE, 'r') as zip_ref:
#     # Use tqdm for a progress bar during extraction
#     for member in tqdm(zip_ref.infolist(), desc='Extracting '):
#         try:
#             zip_ref.extract(member, EXTRACTED_DATA_DIR)
#         except zipfile.error as e:
#             print(f"Error extracting {member.filename}: {e}")
#             continue
print("Extraction complete.")

# Verify and set the actual root directory within the extracted data
# The zip might extract into a subdirectory named 'PlantVillage'
extracted_contents = os.listdir(EXTRACTED_DATA_DIR)
if 'PlantVillage' in extracted_contents and os.path.isdir(os.path.join(EXTRACTED_DATA_DIR, 'PlantVillage')):
    ORIGINAL_DATA_ROOT = os.path.join(EXTRACTED_DATA_DIR, 'PlantVillage')
else:
    # If not 'PlantVillage', assume the class folders are directly under EXTRACTED_DATA_DIR
    ORIGINAL_DATA_ROOT = EXTRACTED_DATA_DIR

print(f"Original dataset root for splitting (containing class folders): {ORIGINAL_DATA_ROOT}")
if not os.path.exists(ORIGINAL_DATA_ROOT) or not os.listdir(ORIGINAL_DATA_ROOT):
    raise FileNotFoundError(f"ERROR: Original data root '{ORIGINAL_DATA_ROOT}' not found or is empty after extraction. Please check the zip content.")

Extraction complete.
Original dataset root for splitting (containing class folders): /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_Raw


In [11]:
# @title 3. Create Train, Validation, Test Subset
# This cell creates the stratified train/validation/test split into the desired structure.

# Clear existing subset directory if it exists to ensure a clean split
if os.path.exists(SUBSET_OUTPUT_DIR):
    print(f"Removing existing subset directory: {SUBSET_OUTPUT_DIR}")
    shutil.rmtree(SUBSET_OUTPUT_DIR)
os.makedirs(SUBSET_OUTPUT_DIR, exist_ok=True) # Recreate the directory

print(f"\n--- Creating train/val/test split into {SUBSET_OUTPUT_DIR} ---")
# Use split-folders to create the train/val/test split
# ratio = (train, val, test)
# seed for reproducibility
# group_prefix=None is suitable for datasets where class names are directly folder names
ratio(
    ORIGINAL_DATA_ROOT,
    output=SUBSET_OUTPUT_DIR,
    seed=1337, # Fixed seed for reproducibility
    ratio=(0.7, 0.15, 0.15), # 70% train, 15% validation, 15% test
    group_prefix=None
)

print("\nDataset splitting complete. Generated directory structure (in Google Drive):")
# List the contents of the newly created train/val/test directories
for split_folder in ['train', 'val', 'test']:
    path = os.path.join(SUBSET_OUTPUT_DIR, split_folder)
    print(f"\nContents of {path}:")
    if os.path.exists(path):
        # List first few class subdirectories to show structure
        class_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
        print(f"  Total classes: {len(class_dirs)}")
        print("  Sample class directories:", sorted(class_dirs)[:5], "...")
    else:
        print(f"  {split_folder} directory not found.")


--- Creating train/val/test split into /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_Subset ---


Copying files: 54310 files [26:14, 34.50 files/s]



Dataset splitting complete. Generated directory structure (in Google Drive):

Contents of /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_Subset/train:
  Total classes: 40
  Sample class directories: ['.git', 'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy'] ...

Contents of /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_Subset/val:
  Total classes: 40
  Sample class directories: ['.git', 'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy'] ...

Contents of /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_Subset/test:
  Total classes: 40
  Sample class directories: ['.git', 'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy'] ...


In [12]:
# @title 4. Verify Data Counts (Optional but Recommended)
# This cell helps verify the number of images in each split for sanity check.

print("\n--- Verifying Image Counts in Each Split ---")
for split in ['train', 'val', 'test']:
    split_path = os.path.join(SUBSET_OUTPUT_DIR, split)
    if os.path.exists(split_path):
        total_images = 0
        num_classes = 0
        print(f"\n{split.upper()} set located at: {split_path}")
        for class_name in sorted(os.listdir(split_path)):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                num_images = len(os.listdir(class_path))
                # print(f"  - {class_name}: {num_images} images") # Uncomment for verbose class-wise counts
                total_images += num_images
                num_classes += 1
        print(f"Total images in {split.upper()} set: {total_images} across {num_classes} classes.")
    else:
        print(f"Warning: {split.upper()} directory not found at {split_path}")

print("\nModule 1 MVP data preparation complete! The structured dataset is ready for training, persistently stored in Google Drive.")


--- Verifying Image Counts in Each Split ---

TRAIN set located at: /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_Subset/train
Total images in TRAIN set: 38001 across 40 classes.

VAL set located at: /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_Subset/val
Total images in VAL set: 8128 across 40 classes.

TEST set located at: /content/drive/MyDrive/AgroAI_Project_Data/module1_edge_ai/data/PlantVillage_Subset/test
Total images in TEST set: 8181 across 40 classes.

Module 1 MVP data preparation complete! The structured dataset is ready for training, persistently stored in Google Drive.
